<a href="https://colab.research.google.com/github/remixwithkj/Backupmac/blob/main/clustering/Content_Based_Recsys_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading the Book Dataset

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import ast

In [3]:
books_df = pd.read_csv('https://raw.githubusercontent.com/manaranjanp/ISB_MLUL/main/clustering/books_summary.csv.zip', index_col=[0])

In [4]:
books_df.head(10)

,book_name,summaries,categories
0,The Highly Sensitive Person,is a self-assessment guide and how-to-live te...,science
1,Why Has Nobody Told Me This Before?,is a collection of a clinical psychologist’s ...,science
2,The Midnight Library,"tells the story of Nora, a depressed woman in...",science
3,Brave New World,presents a futuristic society engineered perf...,science
4,1984,is the story of a man questioning the system ...,science
5,Stolen Focus,explains why our attention spans have been dw...,science
6,The Life-Changing Science of Detecting Bullshit,teaches its readers how to avoid falling for ...,science
7,Dopamine Nation,talks about the importance of living a balance...,science
8,The Art of Statistics,is a non-technical book that shows how statis...,science
9,No Self No Problem,is a provocative read about the implications ...,science


In [5]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5201 entries, 0 to 5244
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   book_name   5201 non-null   object
 1   summaries   5194 non-null   object
 2   categories  5201 non-null   object
dtypes: object(3)
memory usage: 162.5+ KB


In [6]:
books_df.shape

(5201, 3)

In [7]:
books_df.categories.value_counts()

,count
categories,
psychology,595
productivity,485
motivation,434
happiness,424
work,372
business,363
mindfulness,322
relationships,275
communication,236


In [8]:
books_df.dropna(subset=['summaries'], inplace=True)

In [9]:
books_df.drop_duplicates(subset=['summaries'], inplace=True)

In [10]:
books_df = books_df.reset_index(drop=True)

In [11]:
books_df

,book_name,summaries,categories
0,The Highly Sensitive Person,is a self-assessment guide and how-to-live te...,science
1,Why Has Nobody Told Me This Before?,is a collection of a clinical psychologist’s ...,science
2,The Midnight Library,"tells the story of Nora, a depressed woman in...",science
3,Brave New World,presents a futuristic society engineered perf...,science
4,1984,is the story of a man questioning the system ...,science
...,...,...,...
1222,Better Than Before,breaks down the latest research on how to bre...,work
1223,The Happiness Hypothesis,is the most thorough analysis of how you can ...,work
1224,Rich Dad Poor Dad,"tells the story of a boy with two fathers, on...",work
1225,The Ruthless Elimination Of Hurry,"will teach you how to slow down, relax, and l...",mindfulness


## Vectorizing the Summaries

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
# Create tf-idf vectors using the overview column
# remove stop words
# remove sparse words
# remove highky repetitive words

tfidf_vectorizer = TfidfVectorizer(stop_words='english',
                                   min_df = 5,
                                   max_df = 0.8)

In [14]:
tfidf_matrix = tfidf_vectorizer.fit_transform(books_df['summaries'])

In [15]:
tfidf_matrix.shape

(1227, 900)

In [16]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray())

In [17]:
vocabs = list(tfidf_vectorizer.vocabulary_.keys())

In [18]:
tfidf_df.columns = vocabs

In [19]:
from random import sample

sample_vocabs = sample(vocabs,10)

In [20]:
tfidf_df.sample(10)[sample_vocabs]

,religion,identifying,lasting,failure,physics,basis,debunks,want,solutions,got
993,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
301,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1008,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
830,0.0,0.0,0.283631,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1052,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
439,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1218,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
682,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
795,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Computing Pairwise Distances

In [21]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

In [28]:
# 1- cosine to ensure that original measure is retained since pair_wise distance returns 1 - value originally

books_sim = 1 - pairwise_distances( tfidf_matrix, metric="cosine" )

In [29]:
books_sim_df = pd.DataFrame( books_sim )

In [30]:
books_sim_df.head( 10 )

,0,1,2,3,4,5,6,7,8,9,...,1217,1218,1219,1220,1221,1222,1223,1224,1225,1226
0,1.000000,0.0,0.000000,0.000000,0.000000,0.077582,0.037083,0.000000,0.000000,0.07213,...,0.000000,0.000000,0.043648,0.000000,0.000000,0.0,0.000000,0.000000,0.105632,0.000000
1,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.081296,0.000000
2,0.000000,0.0,1.000000,0.000000,0.076252,0.078215,0.000000,0.023923,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.079510,0.000000,0.0,0.022141,0.164250,0.034685,0.000000
3,0.000000,0.0,0.000000,1.000000,0.126309,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.097214,0.000000,0.000000,0.000000,0.000000,0.0,0.226319,0.000000,0.000000,0.000000
4,0.000000,0.0,0.076252,0.126309,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.140918,0.000000,0.000000,0.000000,0.126487,0.0,0.084076,0.059337,0.000000,0.000000
5,0.077582,0.0,0.078215,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.123349,0.000000,0.195044,0.000000,0.0,0.000000,0.000000,0.100842,0.000000
6,0.037083,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.078384,0.157998,0.00000,...,0.078421,0.104375,0.079798,0.086492,0.000000,0.0,0.169283,0.044592,0.000000,0.111226
7,0.000000,0.0,0.023923,0.000000,0.000000,0.000000,0.078384,1.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.271572,0.000000,0.0,0.020419,0.018616,0.031989,0.000000
8,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.157998,0.000000,1.000000,0.00000,...,0.083361,0.110949,0.030791,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
9,0.072130,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


## Finding TopN Similar Books

In [25]:
def get_similar_books( title, topN = 5 ):
    book_idx = books_df[books_df['book_name'] == title].index[0]
    books_df['similarity'] = books_sim_df.iloc[book_idx]
    top_n = books_df.sort_values( ["similarity"], ascending = False )[0:topN+1]
    #print( "Similar Movies to: ", )
    return top_n

### Finding Similar Books

 - The Bitcoin Standard
 - Measure What Matters
 - The Happiness Hypothesis

In [26]:
books_df[books_df.book_name.str.contains("Mindfulness")]

,book_name,summaries,categories
731,The Miracle of Mindfulness,teaches the ancient Buddhist practice of mind...,happiness


In [32]:
get_similar_books('Chaos Monkeys')

,book_name,summaries,categories,similarity
1190,Chaos Monkeys,is a fun behind-the-scenes look that lifts th...,business,1.000000
354,Masters of Scale,teaches entrepreneurs ways to open up a succe...,economics,0.224305
1187,Founders at Work,shows you how to start a successful business ...,business,0.221890
870,The One-Page Financial Plan,"is a refreshing, fun look at personal finance...",money,0.207927
1194,Start-Up Nation,"explains how a tiny, controversial, political...",business,0.206641
1020,Losing My Virginity,details Richard Branson’s meteoric rise to su...,productivity,0.191108


In [ ]:
books_df.sample(20, random_state = 100)

,book_name,summaries,categories,similarity
531,No-Drama Discipline,is a refreshing approach to parenting that lo...,relationships,0.000000
715,Q,uit Like A Millionaire,happiness,0.000000
530,Executive Presence,is an actionable guide to the essential compo...,relationships,0.000000
629,The Comfort Book,explores how depression feels like and its ef...,happiness,0.118908
520,The Secret Life of Pronouns,is a collection of research and case studies ...,relationships,0.000000
830,Get A Financial Life,shows those that are new to managing money ho...,money,0.000000
560,Unlimited Power,"is a self-help classic, which breaks down how...",relationships,0.000000
634,The Burnout Fix,delivers practical advice on how to thrive in...,happiness,0.000000
482,Battle Hymn Of The Tiger Mother,opens your eyes to the potential benefits of ...,relationships,0.000000
985,Mind Gym,explains why the performance of world-class a...,productivity,0.000000


In [ ]:
get_similar_books( 'Executive Presence' )

,book_name,summaries,categories,similarity
530,Executive Presence,is an actionable guide to the essential compo...,relationships,1.000000
285,The Myth Of The Strong Leader,"reveals why being a bold, charismatic leader ...",politics,0.246826
715,Q,uit Like A Millionaire,happiness,0.242532
1211,Pitch Perfect,is the ultimate guide to becoming more effici...,communication,0.223045
114,Exploring The World Of Lucid Dreaming,is a practical guide to dreaming consciously ...,science,0.215610
91,Cosmos,will make you smarter by teaching you the bas...,science,0.211026
